<a href="https://colab.research.google.com/github/Rogerio-mack/IA_2024S2/blob/main/AI_2024S2_Lab3_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

# Lab: Seleção de Atributos, Hiperparâmetros e ML Regressor





# Caso: **Classificação de Tipos de Vidro de Reciclagem**

Nossa base de dados apresenta vidros de reciclagem separados nas categorias:

* **C = Vidros de Construção**
* **V = Vidros de Veículos**
* **O = Outros**

de acordo com sua composição.

Siga o roteiro de tarefas abaixo para fazer uma seleção de atributos e modelos para a predição de novos casos das classes de vidros para reciclagem.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Aquisição dos dados

In [ ]:
df = pd.read_csv('https://github.com/Rogerio-mack/IMT_CD_2024/raw/refs/heads/main/data/recycle_glasses.csv')
display(df.head())
df_cases = pd.read_csv('https://github.com/Rogerio-mack/IMT_CD_2024/raw/refs/heads/main/data/recycle_glasses_cases.csv')
display(df_cases.head())

,Pb,Na,Mg,Al,Si,K,Ca,B,Fe,Type of glass
0,1.52,13.64,D,1.10,71.78,0.06,8.75,0.0,0.0,C
1,1.52,13.89,D,1.36,72.73,0.48,7.83,0.0,0.0,C
2,1.52,13.53,C,1.54,72.99,0.39,7.78,0.0,0.0,C
3,1.52,13.21,D,1.29,72.61,0.57,8.22,0.0,0.0,C
4,1.52,13.27,D,1.24,73.08,0.55,8.07,0.0,0.0,C


,Pb,Na,Mg,Al,Si,K,Ca,B,Fe
0,1.52,12.81,C,1.48,73.89,0.60,8.12,0.0,0.01
1,1.52,12.89,C,1.52,74.10,0.67,7.83,0.0,0.01
2,1.52,12.90,D,1.19,73.44,0.60,8.43,0.0,0.01
3,1.52,13.33,B,1.52,73.04,0.58,8.79,0.0,0.01
4,1.51,13.81,B,3.50,70.89,1.68,5.87,2.2,0.01


# Ex1. Conjunto de Treinamento e Teste, Label Encode (**RESOLVIDO**)

Separe os conjuntos de Treinamento e Teste e faça o **Label Encode** dos dados. **Estratifique** os dados de treinamento e teste pela variável algo. Empregue 20% de dados de teste e o `random_state=42`.

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Type of glass', axis=1)
y = df['Type of glass']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

X_train['Mg'] = le.fit_transform(X_train['Mg'])
X_test['Mg'] = le.transform(X_test['Mg'])

In [ ]:
#@markdown sanity check, must be **True**
X_train.sum().sum() == 16433.35

True

**Não empregue quaisquer outras transformações nos dados**.

# Ex2. Selecione os Atributos

Para a construção do modelo, exclua os 3 atributos com menor ganho de informação (equivalente à informação mútua no scikit-learn).  

O código abaixo já mostra o ganho de informação dos atributos. Você apenas tem que alterar os dados para atender o critério dos atributos que devem ser usados.

In [ ]:
from sklearn.feature_selection import mutual_info_classif

# Calculate mutual information
mutual_info = mutual_info_classif(X_train, y_train)

# Create a DataFrame to store feature names and their mutual information scores
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Mutual Information': mutual_info})

# Sort the DataFrame by mutual information in descending order
feature_importance_df = feature_importance_df.sort_values('Mutual Information', ascending=False)

# Display the feature importance
print(feature_importance_df)

  Feature  Mutual Information
3      Al            0.183147
2      Mg            0.174214
7       B            0.156692
5       K            0.151825
6      Ca            0.120905
1      Na            0.118984
4      Si            0.033699
0      Pb            0.022259
8      Fe            0.000000


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

select_features = SelectKBest(mutual_info_classif, k=6).fit(X_train, y_train)
list(X_train.columns[select_features.get_support()])

['Na', 'Mg', 'Al', 'K', 'Ca', 'B']

In [ ]:
feature_importance_df['Feature'].values[0:-3]

array(['Al', 'Mg', 'B', 'K', 'Ca', 'Na'], dtype=object)

In [ ]:
X_train = X_train[feature_importance_df['Feature'].values[0:-3]]
X_test = X_test[feature_importance_df['Feature'].values[0:-3]]

In [ ]:
#@markdown sanity check, must be **True**
X_train.sum().sum() == 4257.84

True

# Ex3. Decision Tree

Aplique um modelo `DecisionTreeClassifier(random_state=42)` fazendo a escolha com o `GridSearchCV` dos melhores **hiperparâmetros**:

* Profundidade das Árvores: de 5 a 10
* Critério de ganho de informação dos nós: índice `gini`, `entropia` e `logloss`

Empregue como métrica para o melhor modelo o **`accuracy`** e **10 partições de cross validation**. Ao final obtenha os parâmetros, o classification report e score do melhor modelo.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

base_estimator = DecisionTreeClassifier(random_state=42)

param_grid = {
    'max_depth': range(5, 11),
    'criterion': ['gini','entropy','log_loss']
}

clf = GridSearchCV(base_estimator, param_grid, cv=10, scoring='accuracy')

clf.fit(X_train, y_train)

print(f'best estimator: {clf.best_estimator_} with score {clf.best_score_:.3f}')
print()

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'accuracy score in X_test: {clf.score(X_test, y_test):.3f}')

print()
display(pd.DataFrame(clf.cv_results_)[['params','mean_test_score']])

best estimator: DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=42) with score 0.835

              precision    recall  f1-score   support

           C       0.93      0.86      0.89        29
           O       1.00      1.00      1.00         3
           V       0.64      0.78      0.70         9

    accuracy                           0.85        41
   macro avg       0.85      0.88      0.86        41
weighted avg       0.87      0.85      0.86        41

accuracy score in X_test: 0.854

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_criterion', 'param_max_depth', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'split5_test_score', 'split6_test_score', 'split7_test_score', 'split8_test_score', 'split9_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])



,params,mean_test_score
0,"{'criterion': 'gini', 'max_depth': 5}",0.816176
1,"{'criterion': 'gini', 'max_depth': 6}",0.816544
2,"{'criterion': 'gini', 'max_depth': 7}",0.793015
3,"{'criterion': 'gini', 'max_depth': 8}",0.773162
4,"{'criterion': 'gini', 'max_depth': 9}",0.773529
5,"{'criterion': 'gini', 'max_depth': 10}",0.767279
6,"{'criterion': 'entropy', 'max_depth': 5}",0.834926
7,"{'criterion': 'entropy', 'max_depth': 6}",0.822794
8,"{'criterion': 'entropy', 'max_depth': 7}",0.766912
9,"{'criterion': 'entropy', 'max_depth': 8}",0.780515


```
best estimator: DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42) with score 0.829

              precision    recall  f1-score   support

           C       0.93      0.93      0.93        29
           O       1.00      1.00      1.00         3
           V       0.78      0.78      0.78         9

    accuracy                           0.90        41
   macro avg       0.90      0.90      0.90        41
weighted avg       0.90      0.90      0.90        41

accuracy score in X_test: 0.902

best estimator: DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=42) with score 0.841

              precision    recall  f1-score   support

           C       0.93      0.86      0.89        29
           O       1.00      1.00      1.00         3
           V       0.64      0.78      0.70         9

    accuracy                           0.85        41
   macro avg       0.85      0.88      0.86        41
weighted avg       0.87      0.85      0.86        41

accuracy score in X_test: 0.854

best estimator: DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=42) with score 0.835

              precision    recall  f1-score   support

           C       0.93      0.86      0.89        29
           O       1.00      1.00      1.00         3
           V       0.64      0.78      0.70         9

    accuracy                           0.85        41
   macro avg       0.85      0.88      0.86        41
weighted avg       0.87      0.85      0.86        41

accuracy score in X_test: 0.854

best estimator: DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=42) with score 0.829

              precision    recall  f1-score   support

           C       0.93      0.86      0.89        29
           O       1.00      1.00      1.00         3
           V       0.64      0.78      0.70         9

    accuracy                           0.85        41
   macro avg       0.85      0.88      0.86        41
weighted avg       0.87      0.85      0.86        41

accuracy score in X_test: 0.854
```

# Ex4. KNeighbors

Aplique um modelo `KNeighborsClassifier()` fazendo a escolha com o `GridSearchCV` dos melhores **hiperparâmetros**:

* Número de vizinhos: de 5 a 10
* Função distância empregada: a distância padrão do modelo e a distância de `hamming`

Empregue como métrica para o melhor modelo o **`accuracy`** e **10 partições de cross validation**. Ao final obtenha os parâmetros, o classification report e score do melhor modelo.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

base_estimator = KNeighborsClassifier()

param_grid = {
    'n_neighbors': range(5,11),
    'metric': ['minkowski','hamming']
}

clf = GridSearchCV(base_estimator, param_grid, cv=10, scoring='accuracy')

clf.fit(X_train, y_train)

print(f'best estimator: {clf.best_estimator_} with score {clf.best_score_:.3f}')
print()

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print(f'accuracy score in X_test: {clf.score(X_test, y_test):.3f}')

print()
display(pd.DataFrame(clf.cv_results_)[['params','mean_test_score']])

best estimator: KNeighborsClassifier(n_neighbors=6) with score 0.841

              precision    recall  f1-score   support

           C       0.91      1.00      0.95        29
           O       1.00      0.67      0.80         3
           V       0.86      0.67      0.75         9

    accuracy                           0.90        41
   macro avg       0.92      0.78      0.83        41
weighted avg       0.90      0.90      0.90        41

accuracy score in X_test: 0.902

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_metric', 'param_n_neighbors', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'split5_test_score', 'split6_test_score', 'split7_test_score', 'split8_test_score', 'split9_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])



,params,mean_test_score
0,"{'metric': 'minkowski', 'n_neighbors': 5}",0.828309
1,"{'metric': 'minkowski', 'n_neighbors': 6}",0.840809
2,"{'metric': 'minkowski', 'n_neighbors': 7}",0.840809
3,"{'metric': 'minkowski', 'n_neighbors': 8}",0.834559
4,"{'metric': 'minkowski', 'n_neighbors': 9}",0.822426
5,"{'metric': 'minkowski', 'n_neighbors': 10}",0.834559
6,"{'metric': 'hamming', 'n_neighbors': 5}",0.786765
7,"{'metric': 'hamming', 'n_neighbors': 6}",0.786765
8,"{'metric': 'hamming', 'n_neighbors': 7}",0.804044
9,"{'metric': 'hamming', 'n_neighbors': 8}",0.792279


```
best estimator: KNeighborsClassifier(n_neighbors=6) with score 0.841

              precision    recall  f1-score   support

           C       0.91      1.00      0.95        29
           O       1.00      0.67      0.80         3
           V       0.86      0.67      0.75         9

    accuracy                           0.90        41
   macro avg       0.92      0.78      0.83        41
weighted avg       0.90      0.90      0.90        41

accuracy score in X_test: 0.902
```

# Ex5. Predict

Faça a predição dos novos casos com base no melhor modelo obtido.

**Dica**: Você terá de aplicar a mesma transformação aplicada antes para os valores de `Mg`.

In [ ]:
df_cases['Mg'] = le.transform(df_cases['Mg'])
df_cases = df_cases[feature_importance_df['Feature'].values[0:-3]]
df_cases.head()

,Al,Mg,B,K,Ca,Na
0,1.48,2,0.0,0.60,8.12,12.81
1,1.52,2,0.0,0.67,7.83,12.89
2,1.19,3,0.0,0.60,8.43,12.90
3,1.52,1,0.0,0.58,8.79,13.33
4,3.50,1,2.2,1.68,5.87,13.81


In [ ]:
clf.predict(df_cases)

array(['C', 'C', 'C', 'C', 'V', 'C', 'V'], dtype=object)

# Ex6. KNeighbors Regressor

Adapte o código de seleção do modelo `KNeighbors Classifier` para implementar uma regressão não linear para estimar o `petal_length` com base nos demais atributos numéricos. Obtenha o `score` do modelo ($R2$) sobre o conjunto de teste.

**Dica**: É um problema agora de regressão, a métrica não é, portanto, a acuracidade, não existe o classification_report, ... mas o `clf.score(X_test, y_test)` devolve `score` do modelo ($R2$) corretamente.

In [ ]:
df = sns.load_dataset('iris')
display(df.head())



,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

X = df[['sepal_length', 'sepal_width', 'petal_width']]
y = df['petal_length']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

base_estimator = KNeighborsRegressor()

param_grid = {
    'n_neighbors': range(5,11),
    'metric': ['minkowski','hamming']
}

clf = GridSearchCV(base_estimator, param_grid, cv=10)

clf.fit(X_train, y_train)

print(f'best estimator: {clf.best_estimator_} with score {clf.best_score_:.3f}')
print()

print(f'accuracy score in X_test: {clf.score(X_test, y_test):.3f}')


best estimator: KNeighborsRegressor() with score 0.990

accuracy score in X_test: 0.990
